In [1]:
!pip install --upgrade twarc
!pip install twarc-csv --user
from google.colab import drive
drive.mount('/content/drive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 60 kB 5.0 MB/s 
     |████████████████████████████████| 106 kB 19.0 MB/s 
  Created wheel for configobj: filename=configobj-5.0.6-py3-none-any.whl size=34547 sha256=7f5feb9b1fb5bdd6f74c5dbb2a1f8e97001ce65f3c6de9667df6af54f1f67df8
  Stored in directory: /root/.cache/pip/wheels/0d/c4/19/13d74440f2a571841db6b6e0a273694327498884dafb9cf978
Successfully built configobj
  Attempting uninstall: humanize
    Found existing installation: humanize 0.5.1
    Uninstalling humanize-0.5.1:
      Successfully uninstalled humanize-0.5.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for twarc-csv: filename=twarc_csv-0.5.2-py3-none-any.whl size=9668 sha256=990db7c9e4a1f6b66127f026b19a4a04f2fcace55750c4aa2f074c39be409dc6
  Stored in directory: /root/.cache/pip/wheels/8b/da/29/686e584f0162297399d799af5

In [2]:
import nltk, numpy, pandas, os, sys, random, time, twarc
import matplotlib.pyplot as plt
import pandas as pd
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

BEARER: AAAAAAAAAAAAAAAAAAAAAFkmfgEAAAAADcgLX%2B3II8bI4oZ7dDWTGfqEsis%3DckGXQZEmb9y5fkhZR34eXvdatbiWSSg4vCxZla4pmfGXndZCD2   

API KEY: hPOI2LgsaW5vylwt0AmCXLHXA   

API SECRET: mcNe4tlYUKhmDOWmB4ixZN7qYSfTD71Z6KAU08rxtletApPxnr  

ACCESS TOKEN: 1430421232239796227-BctF5asyBRXL3xoLioeq7Q0iWeiQV0  

ACCESS SECRET: OK5KkazmfZSUWJoZ142trqFDac350T7H2bt7K1WxmTD8N




In [3]:
!twarc2 configure


👋  Hi I don't see a configuration file yet, so let's make one.

Please follow these steps:

1. visit https://developer.twitter.com/en/portal/
2. create a project and an app
3. go to your Keys and Tokens and generate your keys

Please enter your Bearer Token (leave blank to skip to API key configuration): AAAAAAAAAAAAAAAAAAAAAFkmfgEAAAAADcgLX%2B3II8bI4oZ7dDWTGfqEsis%3DckGXQZEmb9y5fkhZR34eXvdatbiWSSg4vCxZla4pmfGXndZCD2
(Optional) Add API keys and secrets for user mode authentication [y or n]? y
Please enter your API key: hPOI2LgsaW5vylwt0AmCXLHXA
Please enter your API secret: mcNe4tlYUKhmDOWmB4ixZN7qYSfTD71Z6KAU08rxtletApPxnr

How would you like twarc to obtain your user keys?

1) generate access keys by visiting Twitter
2) manually enter your access token and secret

Please enter your choice [1 or 2] 2
Enter your Access Token: 12Avzkh84Z7Ng3HbQGfo1iu2KHK4gShWH4LiaELbxhbppjDQ
Enter your Access Token Secret: mcNe4tlYUKhmDOWmB4ixZN7qYSfTD71Z6KAU08rxtletApPxnr ACCESS TOKEN: 143042123223979

In [8]:
!twarc2 search "russia OR ukraine OR donbass OR luhansk OR crimea OR kherson OR kharkiv OR putin OR zelensky OR zaporizhzhia" ./ukrain_REST.jsonl --start-time 2022-09-15 --end-time 2022-09-21 --limit 1000000
!twarc2 stream-rules add "donbass"
!twarc2 stream-rules add "russia"
!twarc2 stream-rules add "ukraine"
!twarc2 stream-rules add "luhansk"
!twarc2 stream-rules add "crimea"
!twarc2 stream-rules add "kherson"
!twarc2 stream-rules add "kharkiv"
!twarc2 stream-rules add "putin"
!twarc2 stream-rules add "zelensky"
!twarc2 stream-rules add "zaporizhzhia"
!twarc2 stream > ukrain_STREAM.jsonl

  1% 5242782/518400000 [13:47<22:30:05, 6334.85it/s]
⚡ Invalid 'start_time':'2022-09-15T00:00Z'. 'start_time' must be on or after 2022-09-15T00:03Z


In [16]:
#!twarc2 flatten /content/drive/MyDrive/ukraine_REST.jsonl /content/drive/MyDrive/ukraine_REST_flat.jsonl
#!twarc2 flatten /content/drive/MyDrive/ukraine_STREAM.jsonl /content/drive/MyDrive/ukraine_STREAM_flat.jsonl

100% 633M/633M [01:19<00:00, 8.34MB/s]
⚡ Unterminated string starting at: line 1 column 3807 (char 3806)


In [36]:
!pip3 install twarc-network
!twarc2 network /content/drive/MyDrive/ukraine_STREAM_flat.jsonl --format csv network.csv --min-component-size 30


Aborted!


In [4]:
stream_df = pd.read_csv('/content/drive/MyDrive/network_stream.csv', names=['Source', 'Target', 'Weight', 'Retweet', 'Reply', 'Quote', 'Mention'])
stream_df = stream_df.drop(['Mention'], axis=1)
stream_df['Weight'] = stream_df['Weight'].map(lambda y: [int(x) for x in y.split() if x.isdigit()][0])
stream_df['Retweet'] = stream_df['Retweet'].map(lambda y: [int(x) for x in y.split() if x.isdigit()][0])
stream_df['Reply'] = stream_df['Reply'].map(lambda y: [int(x) for x in y.split() if x.isdigit()][0])
stream_df['Quote'] = stream_df['Quote'].map(lambda y: [int(x) for x in y.split() if x.isdigit()][0])

Mounted at /content/drive


In [8]:
net = nx.from_pandas_edgelist(stream_df, source="Source", target="Target", edge_attr='Weight', create_using=nx.DiGraph())
cugraph.draw(net, with_labels=True, node_size=60,font_size=8)
plt.figure(figsize=(200,200)) 
plt.show()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached cugraph-0.6.1.post1.tar.gz (1.1 kB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for cugraph
  Running setup.py clean for cugraph
Failed to build cugraph
  error: subprocess-exited-with-error
  
  × Running setup.py install for cugraph did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Running setup.py install for cugraph ... error
error: legacy-install-failure

× Encountered error while trying to install package.
╰─> cugraph

note: This is an issue with the package mentioned above, not pip.
hint: See above 

ModuleNotFoundError: ignored